In [29]:
# Use me

import time
import random
import requests
from bs4 import BeautifulSoup
import json
from pathlib import Path

from city_code import CITY_CODE, REGION

class House591Spider():
    def __init__(self):
        self.headers = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36 Edg/88.0.705.68',
        }

    def fetch_all_brief_rental_object(self, session, headers, county, json_object):
        result = []
        firstRow = 0

        total_count = int(json_object['records'].replace(',', ""))
        print(f"county={county}, total_count={total_count}")


        while len(result) < total_count:
            each_page_url = f"https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region={county}&recom_community=1&firstRow={firstRow}"
            response = session.get(each_page_url, headers=headers)

            # print(f"self-header: {self.headers}")
            # print(f"normal-header: {headers}")

            result += response.json()['data']['data']
            # print(result)
            firstRow = len(result)


        return result

    def fetch_a_detail_rental_house(self, postid):
        
        # 紀錄 Cookie 取得 X-CSRF-TOKEN
        s = requests.Session()
        url = 'https://rent.591.com.tw/'
        r = s.get(url, headers=self.headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        token_item = soup.select_one('meta[name="csrf-token"]')

        headers = self.headers.copy()
        # 如果是詳細資訊，還需要另外多帶pc和deviceid到header裡
        headers['X-CSRF-TOKEN'] = token_item.get('content')
        headers['deviceid'] = s.cookies.get_dict()['T591_TOKEN']

        headers['device'] = 'pc'
        print(s.cookies.get_dict())

        result = {}

        url = f"https://bff.591.com.tw/v1/house/rent/detail?id={postid}"
        print(f"fetch a detail rantal house")
        print(headers, postid )

        response = requests.get(url=url, headers=headers)
        print(response.json())
    
    
    
    def search(self, filter_params=None, sort_params=None, want_page=1):
        """ 搜尋房屋

        :param filter_params: 篩選參數
        :param sort_params: 排序參數
        :param want_page: 想要抓幾頁
        :return total_count: requests 房屋總數
        :return house_list: requests 搜尋結果房屋資料
        """
        total_count = 0
        house_list = []
        page = 0

        # 紀錄 Cookie 取得 X-CSRF-TOKEN
        s = requests.Session()
        url = 'https://rent.591.com.tw/'
        r = s.get(url, headers=self.headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        token_item = soup.select_one('meta[name="csrf-token"]')

        headers = self.headers.copy()
        headers['X-CSRF-TOKEN'] = token_item.get('content')

        # print(headers)
        # 搜尋房屋
        url = 'https://rent.591.com.tw/home/search/rsList'
        params = 'is_format_data=1&is_new_list=1&type=1&recom_community=1'
        if filter_params:
            # 加上篩選參數，要先轉換為 URL 參數字串格式
            params += ''.join([f'&{key}={value}' for key, value, in filter_params.items()])
        else:
            params += '&region=1&kind=0'
        # 在 cookie 設定地區縣市，避免某些條件無法取得資料
        # 這行是關鍵
        s.cookies.set('urlJumpIp', filter_params.get('region', '1') if filter_params else '1', domain='.591.com.tw')
        # s.cookies.set('urlJumpIp', '1', domain='.591.com.tw')
        
        # s.cookies.set('urlJumpIp', '11', domain='.591.com.tw')
        

        # # 排序參數
        # if sort_params:
        #     params += ''.join([f'&{key}={value}' for key, value, in sort_params.items()])
        # print(params)
        # while page < want_page:
        #     params += f'&firstRow={page*30}'
        #     r = s.get(url, params=params, headers=headers)
        #     if r.status_code != requests.codes.ok:
        #         print('請求失敗', r.status_code)
        #         break
        #     page += 1

        #     data = r.json()
        #     total_count = data['records']
        #     house_list.extend(data['data']['data'])
        #     # 隨機 delay 一段時間
        #     time.sleep(random.uniform(2, 5))

        r = s.get(url, params=params, headers=headers)
        # r = s.get(url, headers=headers)
        # print(r.json()['records'])
        rental_houses = self.fetch_all_brief_rental_object(session=s, headers=headers, county=filter_params['region'], json_object=r.json())
        return rental_houses



    # def get_house_detail(self, house_id):
    #     """ 房屋詳情

    #     :param house_id: 房屋ID
    #     :return house_detail: requests 房屋詳細資料
    #     """
    #     # 紀錄 Cookie 取得 X-CSRF-TOKEN, deviceid
    #     s = requests.Session()
    #     url = f'https://rent.591.com.tw/home/{house_id}'
    #     r = s.get(url, headers=self.headers)
    #     soup = BeautifulSoup(r.text, 'html.parser')
    #     token_item = soup.select_one('meta[name="csrf-token"]')

    #     headers = self.headers.copy()
    #     headers['X-CSRF-TOKEN'] = token_item.get('content')
    #     headers['deviceid'] = s.cookies.get_dict()['T591_TOKEN']
    #     # headers['token'] = s.cookies.get_dict()['PHPSESSID']
    #     headers['device'] = 'pc'

    #     url = f'https://bff.591.com.tw/v1/house/rent/detail?id={house_id}'
    #     r = s.get(url, headers=headers)
    #     if r.status_code != requests.codes.ok:
    #         print('請求失敗', r.status_code)
    #         return
    #     house_detail = r.json()['data']
    #     return house_detail


def write_brief_json(path, data, county_id):
    
    with open(Path(path)/f'{county_id}-brief.json', 'w') as outfile:
        json.dump(data, outfile)


## 抓brief 資料


In [ ]:
if __name__ == "__main__":
    house591_spider = House591Spider()

    # # 篩選條件
    # filter_params = {
    #     'region': '12',  # (地區) 台北
    #     # 'searchtype': '4',  # (位置1) 按捷運
    #     # 'mrtline': '125',  # (位置2) 淡水信義線
    #     # 'mrtcoods': '4198,4163',  # (位置3) 新北投 & 淡水
    #     # 'kind': '2',  # (類型) 獨立套房
    #     # 'multiPrice': '0_5000,5000_10000',  # (租金) 5000元以下 & 5000-10000元
    #     # 'rentprice': '3000,6000',  # (自訂租金範圍) 3000~6000元
    #     # 'multiRoom': '2,3',  # (格局) 2房 & 3房
    #     # 'other': 'near_subway,cook,lease',  # (特色) 近捷運 & 可開伙 & 可短期租賃
    #     # --- 以下要加 showMore=1 ---
    #     # 'showMore': '1',
    #     # 'shape': '3',  # (型態) 透天厝
    #     # 'multiArea': '10_20,20_30,30_40',  # (坪數) 10-20坪 & 20-30坪 & 30-40坪
    #     # 'area': '20,50',  # (自訂坪數範圍) 20~50坪
    #     # 'multiFloor': '2_6',  # (樓層) 2-6層
    #     # 'option': 'cold,washer,bed',  # (設備) 有冷氣 & 有洗衣機 & 床
    #     # 'multiNotice': 'all_sex',  # (須知) 男女皆可
    # }
    # # 排序依據
    # sort_params = {
    #     # 租金由小到大
    #     'order': 'money',  # posttime, area
    #     'orderType': 'desc'  # asc
    # }

    
    # 抓brief
    all_county = REGION['whole']
    for county in all_county:
        filter_params = {'region': str(county)}
        print(county)
        rental_houses = house591_spider.search(filter_params)  # edge case: 連江可能抓不到資料
        write_brief_json(path="./brief-jsons/rental-house/", data=rental_houses, county_id=county)

In [7]:
# Combine all the brief-rental-data

from pathlib import Path
import json
all_json = []

for json_file in Path('./brief-jsons/rental-house').glob('*.json'):
    # print(json_file)
    with open(json_file) as jf:
        # all_json.update(json.load(jf))
        # print(json.load(jf))
        all_json += json.load(jf)

def find_all_columns(json_data):
    columns = set()

    for house in json_data:
        for column in house:
            columns.add(column)

    return columns

columns = find_all_columns(all_json)

In [9]:
import pandas as pd
def create_df(columns, json_data):
    data_to_df = {}

    for column in columns:
        data_to_df[column] = []
    # fill in each data
    for data in json_data:
        # for column in data:
        for column in columns:
            # key is column
            # value is data[column] if exists
            try:
                data_to_df[column].append(str(data[column]))
            except KeyError:
                data_to_df[column].append("")
    data_to_df = pd.DataFrame.from_dict(data_to_df)
    return data_to_df

    
    # return df

df = create_df(columns, all_json)

In [11]:
df.to_csv("./output-file/rental-house/20231024-brief-all-data.csv", encoding='utf-8')

## Using API to fetch the detail rental house object

In [30]:
house591_spider = House591Spider()
postid = 15322990
detail_information = house591_spider.fetch_a_detail_rental_house(postid)


{'PHPSESSID': 'gku7qehu6jfgamkrbsp9sthf1m', 'T591_TOKEN': 'gku7qehu6jfgamkrbsp9sthf1m', 'newUI': '1', 'new_rent_list_kind_test': '1', 'urlJumpIp': '1', 'urlJumpIpByTxt': '%E5%8F%B0%E5%8C%97%E5%B8%82', 'webp': '1', '591_new_session': 'eyJpdiI6ImRjVmk1VjVsWlBaa1RydHIwWlFNb1E9PSIsInZhbHVlIjoiamxkT1MwWmF3cFlIcmM3Vy9uSDJkMkhDZ1M3UnBwZWNZaUFlZ1Fpa0dGZDEzMHhRK0dXWmMyWGhrY0h6VW1TNzI0amt6SUlCN2t6MHBKMkYxeHl2TDVUaWNNekxza3NMMEY3QTNFdVJIakU1V3F1dGlxTFBZeC9rbm5Dc3BCcVMiLCJtYWMiOiI4NDMxYzhmNTAyYTIwZjZmOTFhZWFhYmU4MDA4NDYxNzc0NGUzNjc0NDQ4MGI2NmZhZTQ4ODdiYmFhZTUxYmFjIiwidGFnIjoiIn0%3D'}
fetch a detail rantal house
{'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36 Edg/88.0.705.68', 'X-CSRF-TOKEN': '80QmN63fN98phtzka1X0QNiSDhHHhNJSWzq8Zslm', 'deviceid': 'gku7qehu6jfgamkrbsp9sthf1m', 'device': 'pc'} 15322990
{'status': 1, 'msg': '', 'data': {'breadcrumb': [{'name': '台中市', 'id': 8, 'query': 'region', 'link': '/?region=8'}, 